# 　infercodeのベクトル追加

正解ラベル$$$code2seq$$$inferベクトルという形にへんかんする

In [4]:

import pandas as pd
import numpy as np
from infercode.client.infercode_client import InferCodeClient
import os
import logging

Linux.zip: 13.6MB [00:04, 3.11MB/s]
2022-05-25 17:04:12.093694: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
RAW_DATA_PATH = "./data/raw_data_splitted"
C2S_DATA_PATH = "./data/c2s"
OUTPUT_DIR = "./data/infercode_c2s"

In [7]:
# ここでエラーが出たときはディレクトリに飛んでLinux.zipを解答すること
infercode = InferCodeClient(language="java")
infercode.init_from_config()

universal_model.zip: 368MB [00:16, 22.8MB/s]                           
/home/sdlab/.local/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/sdlab/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2022-05-25 17:17:26.940104: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-25 17:17:26.940628: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-25 17:17:26.975849: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941]

INFO:tensorflow:Restoring parameters from /home/sdlab/.infercode_data/model_checkpoint/universal_model/cnn_tree.ckpt


In [8]:
# inferベクトルの取得
def get_infer_vec(code):
    codeline = code.replace("\t","").replace("\n","")
    vectors = infercode.encode([codeline])
    return vectors[0]

In [17]:
# 特徴量の追加
def add_infer(category:str):
    df = pd.read_csv(f"{RAW_DATA_PATH}/{category}.csv",index_col=0)
    c2vlines = []
    with open(f"{C2S_DATA_PATH}/{category}.c2s") as f:
        c2vlines = f.readlines()
    for i in range(len(c2vlines)):
        beforecode = df.before.values[i]
        vecs = get_infer_vec(beforecode)
        c2vlines[i] = c2vlines[i].replace("  ","").replace("\n","") + "$$$"
        for vec in vecs:
            c2vlines[i] = c2vlines[i] + " " +str(vec)
    f = open(f"{OUTPUT_DIR}/{category}.c2s", "w")
    for c2v in c2vlines:
        f.write(c2v+"\n")

In [24]:
categoryes = ["train","val","test"]
for category in categoryes:
    add_infer(category)